# Image Processing
Ways to do it in JupyterLab

In [ ]:
import IPython.display

In [ ]:
?IPython.display.Image

## display.Image can't take raw data

In [ ]:
import numpy as np
import IPython
IPython.display.Image(np.arange(16*16))

In [ ]:
IPython.display.Image(np.arange(16*16, dtype='uint8').reshape(16,16))

Inspecting the HTML in the browser, we find
`<img src="data:image/png;base64,AAECAwQFB` ... `f4+fr7/P3+/w==
">`

It expects the data in the constructor to be `PNG` it seems

## Make a `PNG` image in-memory

In [ ]:
import PIL

In [ ]:
PIL.__version__

### PIL does late loading of its `Image` class:

In [ ]:
try:
    PIL.Image
except AttributeError as e:
    print(e)

This makes it available:

In [ ]:
from PIL import Image as PILImage

In [ ]:
PIL.Image

### Make a PIL Image from `numpy` array

In [ ]:
img_1 = PIL.Image.fromarray(np.arange(16*16, dtype='uint8').reshape(16,16))

PIL doesn't expose in-memory format conversion. but python can capture it:

In [ ]:
from io import BytesIO
t_png = BytesIO()
img_1.save(t_png, format="png")

In [ ]:
t_png.getbuffer()

In [ ]:
t_di = IPython.display.Image(t_png.getbuffer())
t_di

In [ ]:
display(t_di)

In [ ]:
t2_di = IPython.display.Image(t_png.getbuffer(), width=64, height=32)
t2_di

In [ ]:
img_2 = PIL.Image.fromarray(np.arange(16*16*3, dtype='uint8').reshape(16,16,3))
img_2.getbands()

In [ ]:
t2_png = BytesIO()
img_2.save(t2_png, format="png")

In [ ]:
t2_di = IPython.display.Image(t2_png.getbuffer(), width=64)

In [ ]:
t2_di

In [ ]:
np.arange(0,6,2, dtype='uint8') * 100

In [ ]:
ramp = np.arange(12*16, dtype='uint8').reshape(12,-1)
ramp.shape

In [ ]:
tcramp = np.array([ramp, 2*ramp, 3*ramp])
tcramp.shape

In [ ]:
rtcramp = np.moveaxis(tcramp, 0, -1)
rtcramp.shape

In [ ]:
img_3 = PIL.Image.fromarray(rtcramp)
img_3.getbands(), img_3.size

In [ ]:
t3_png = BytesIO()
img_3.save(t3_png, format="png")

In [ ]:
t3_di = IPython.display.Image(t3_png.getbuffer())

In [ ]:
t3_di

In [ ]:
rtcramp[0,5]

In [ ]:
def d_image_from_pil_image(pim, **kwargs):
    b = BytesIO()
    pim.save(b, format="png")
    return IPython.display.Image(b.getbuffer(), **kwargs)

In [ ]:
def dimage_from_array(a, **kwargs):
    b = BytesIO()
    PIL.Image.fromarray(a).save(b, format="png")
    return IPython.display.Image(b.getbuffer(), **kwargs)

In [ ]:
dimage_from_array(rtcramp)

In [ ]:
image = PIL.Image.fromarray(rtcramp)
image = image.convert('P', palette=PIL.Image.ADAPTIVE, colors=256)
d_image_from_pil_image(image, width=256)

Some CSS modifiers for `img` tags:

    img {
      image-rendering: auto;
      image-rendering: crisp-edges;
      image-rendering: pixelated;
    }

Can do it in the tag, e.g

    <img
         style="image-rendering: pixelated;"
         width="100" height="100"
         src="data:image/png;base64,iVBORw0KGgoAAAA....Ik2kAAAAASUVORK5CYII=">

In [ ]:
d_image_from_pil_image(image, width=256, metadata={'style': 'image-rendering: pixelated'})

In [ ]:
from IPython.display import HTML

In [ ]:
html = """<img 
         style="image-rendering: pixelated;"
         width="200" height="200"
         src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABAAAAAMCAIAAADkharWAAAAJElEQVR4nGNkYGBgZGImAQkoGJCkgWW4asBnxFDxA6UaUBQAAOdQCN0ipdGmAAAAAElFTkSuQmCC
">"""

HTML(html)

In [ ]:
di = dimage_from_array(rtcramp)

In [ ]:
di._repr_png_()

In [ ]:
img_tag_template = \
"""<img alt="{alt}"
        style="image-rendering: pixelated;"
        width="{width}" height="{height}"
        src="data:image/png;base64,{rad64_png}">"""

In [ ]:
kwargs = {'alt': "a test image",
          'unused': 0,
          'width': 320,
          'height': 240,
          'rad64_png': di._repr_png_()}
HTML(img_tag_template.format(**kwargs))

## Using matplotlib

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(rtcramp)
plt.show()

In [ ]:
plt.matshow(rtcramp, fignum=5)
plt.show()

In [ ]:
plt.matshow(rtcramp*2, fignum=5)
plt.show()

# Visualizing neural nets

In [ ]:
from nn import Network, Layer, IdentityLayer, AffineLayer, MapLayer

In [ ]:
net = Network()
net.extend(AffineLayer(8,8))
net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))
net.extend(AffineLayer(8,3))
net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))

In [ ]:
net.state_vector()

In [ ]:
net.layers

In [ ]:
plt.matshow(net.layers[0].M)
plt.show()

In [ ]:
import matplotlib
#?matplotlib.axes.Axes.imshow

In [ ]:
[plt.matshow(np.concatenate((M, np.atleast_2d(b)), axis=0))
        for M, b in ((l.M, l.b) for l in net.layers if hasattr(l, 'M'))]

# `bqplot.GridHeatMap` as framework

In [ ]:
import numpy as np
from IPython.display import display
from bqplot import *

np.random.seed(0)
data = np.random.randn(5,5)
col_sc = ColorScale()
grid_map = GridHeatMap(color=data, scales={'color': col_sc}, 
                       interactions={'click': 'select'},
                       selected_style={'stroke': 'blue', 'stroke-width': 3})
figure=Figure(marks=[grid_map], padding_y=0.0, min_aspect_ratio=1, max_aspect_ratio=1)

from ipywidgets import Output
out = Output()
@out.capture()
def print_event(self, target):
    print(target)
    
# test
print_event(1, 'test output')
grid_map.on_element_click(print_event)

display(figure)
display(out)

In [ ]:
grid_map.color = np.random.randn(6,5)

In [ ]:
[plt.matshow(np.concatenate((M, np.atleast_2d(b)), axis=0))
        for M, b in ((l.M, l.b) for l in net.layers if hasattr(l, 'M'))]

---

# PXImage class
From an array to a `display`-able object that accepts `width` and `height` and `alt` parameters in the constructor.

In [ ]:
class PXImage():
    def __init__(self, a, **kwargs):
        self.a = a
        self.kwargs = kwargs
        self.defaults = {'alt': "unblurred image",
                         'width': a.shape[1],
                         'height': a.shape[0] }
        self.img_tag_template = """<img alt="{alt}"
        style="image-rendering: pixelated;"
        width="{width}" height="{height}"
        src="data:image/png;base64,{rad64_png}">"""
    
    def pil_image_from_array(self, a):
        return PIL.Image.fromarray(a)
    
    @property
    def pil_image(self):
        if not hasattr(self, '_pil_image'):
            self._pil_image = self.pil_image_from_array(self.a)
        return self._pil_image
        
    def d_image_from_pil_image(self, pim, **kwargs):
        b = BytesIO()
        pim.save(b, format="png")
        return IPython.display.Image(b.getbuffer(), **kwargs)
    
    @property
    def d_image(self):
        if not hasattr(self, '_d_image'):
            self._d_image = self.d_image_from_pil_image(self.pil_image)
        return self._d_image
        
    
    def _repr_png_(self):
        return self.d_image._repr_png_()

    def _repr_html_(self):
        d = self.defaults.copy()
        d['rad64_png'] = self._repr_png_()
        d.update(self.kwargs)
        return HTML(self.img_tag_template.format(**d))._repr_html_()

In [ ]:
pxi = PXImage(rtcramp, width=640)

In [ ]:
display(pxi)

In [ ]:
%debug

In [ ]:
d = {'a': 1}
d2 = d.copy()
d['a'] = 0
d, d2

In [ ]:
rtcramp.shape

___